In [2]:
import pandas as pd

In [2]:
def read_markdown_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()

    # Split the content by the markdown table titles
    sections = content.split('##')

    # Create lists to store the titles and tables
    title = []
    table = []

    # Loop through the sections and split the tables from the titles
    for section in sections:
        title.append(section.split('\n')[0])
        table.append(section.split('\n')[1:])

    df = pd.DataFrame(list(zip(title, table)), columns=['Title', 'Table'])  

    return df


markdown_content = read_markdown_file('tables.md')
print(markdown_content)

                                                Title  \
0                                            # Tables   
1                        Table R: The regions in FUND   
2                Table P.FUND: Population; 2000 = 100   
3                Table P.A1B: Population; 2000 = 100.   
4                 Table P.A2: Population; 2000 = 100.   
5                 Table P.B1: Population; 2000 = 100.   
6                 Table P.B2: Population; 2000 = 100.   
7          Table Y.FUND: Per capita income; 2000=1.00   
8          Table Y.A1: Per capita income; 2000 = 1.00   
9          Table Y.A2: Per capita income; 2000 = 1.00   
10         Table Y.B1: Per capita income; 2000 = 1.00   
11         Table Y.B2: Per capita income; 2000 = 1.00   
12    Table AEEI.FUND: Energy efficiency; 2000 = 1.00   
13     Table AEEI.A1B: Energy efficiency; 2000 = 1.00   
14      Table AEEI.A2: Energy efficiency; 2000 = 1.00   
15      Table AEEI.B1: Energy efficiency; 2000 = 1.00   
16      Table AEEI.B2: Energy e

In [7]:
for line in markdown_content.iterrows():

    # Extracting the table name and the table content
    table_name = line[1]['Title']
    file_name = table_name.split(':')[0].strip().replace(' ', '_').lower()
    table = line[1]['Table']

    # Extracting the header (columns) and the data
    header = table[1].split('|')[1:-1]
    print('Header :' +str(header) + str(type(header)))
    data = table[3:-2]
    values = [line.split('|')[1:-1] for line in data]

    try: 

        # export df to csv with FUND regions
        df = pd.DataFrame(values, columns=header)
        df = df.fillna(0)
        df.to_csv(f'output\{file_name}.csv', index=False)
        print(f'Saved output\\fund_{file_name}.csv')

        # export df to csv with WILIAM regions
        wiliam_output = FUND_to_WILIAM(df)
        wiliam_output = wiliam_output.fillna(0)
        wiliam_output.to_excel(f'output\wiliam_{file_name}.xlsx')
    except Exception as e:
        print('Error for table: ', line[1]['Title'], e)
    print(df)

<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:25: SyntaxWarning: invalid escape sequence '\w'
<string>:19: SyntaxWarning: invalid escape sequence '\{'
<>:19: SyntaxWarning: invalid escape sequence '\{'
<>:25: SyntaxWarning: invalid escape sequence '\w'
C:\Users\gabriel.genelot\AppData\Local\Temp\ipykernel_25048\3872861123.py:19: SyntaxWarning: invalid escape sequence '\{'
  df.to_csv(f'output\{file_name}.csv', index=False)
C:\Users\gabriel.genelot\AppData\Local\Temp\ipykernel_25048\3872861123.py:25: SyntaxWarning: invalid escape sequence '\w'
  wiliam_output.to_excel(f'output\wiliam_{file_name}.xlsx')


Header :[]<class 'list'>
Saved output\fund_#_tables.csv
Could not find values for AUSTRIA : WEU
Could not find values for BELGIUM : WEU
Could not find values for BULGARIA : CEE
Could not find values for CROATIA : CEE
Could not find values for CYPRUS : WEU
Could not find values for CZECHIA : nan
Could not find values for DENMARK : WEU
Could not find values for ESTONIA : FSU
Could not find values for FINLAND : WEU
Could not find values for FRANCE : WEU
Could not find values for GERMANY : WEU
Could not find values for GREECE : WEU
Could not find values for HUNGARY : CEE
Could not find values for IRELAND : WEU
Could not find values for ITALY : WEU
Could not find values for LATVIA : FSU
Could not find values for LITHUANIA : FSU
Could not find values for LUXEMBOURG : WEU
Could not find values for MALTA : WEU
Could not find values for NETHERLANDS : WEU
Could not find values for POLAND : CEE
Could not find values for PORTUGAL : WEU
Could not find values for ROMANIA : CEE
Could not find values 

What i want to do : 

- have a table with the regions from WILIAM with the corresponding coefficient from the FUND model

To do so, i can: 

- create a table that has the appropriate form for WILIAM : years in lines, for example, and regions in columns
- for every value, looking for the corresponding value in the FUND table thanks to the .iloc function => something like for col in df.columns: wiliam_df.iloc[year, region] = fund_df.iloc[year, region_in_FUND, where region_in_fund = map_regions('WILIAM', 'FUND', region) is a function that maps a region from model A to model B. 


More specifically, i should

In [17]:
def map_regions(model_A, model_B, region): 
    """
    Maps a region from model A to model B using a correspondence table.

    Parameters:
    - model_A (str): Name of the source model.
    - model_B (str): Name of the target model.
    - region (str): Name of the region to be mapped.

    Returns:
    - output_region (str): Mapped region in model B.

    """

    try: 
        table = pd.read_csv(r'C:\Users\gabriel.genelot\Documents\damage-functions\WILIAM\geography\correspondance.csv')
        table.set_index(model_A, inplace=True)
        output_region = table.loc[region, model_B]
    except Exception as e:
        output_region = f'Region not found. Error: {e}'

    return output_region

In [1]:
test = """
| -    | α           | δ    | β           |
|------|-------------|------|-------------|
| USA  | 0.000120686 | 0.04 | 0.2912144   |
| CAN  | 0.000169725 | 0.04 | 0.063117456 |
| WEU  | 0.000209185 | 0.04 | 0.121209462 |
| JPK  | 1.04096E-05 | 0.04 | 0.114939831 |
| ANZ  | 0.000276264 | 0.21 | 0.116317932 |
| EEU  | 4.58675E-05 | 0.04 | 0.050081393 |
| FSU  | 4.4056E-05  | 0.04 | 0.12684268  |
| MDE  | 1.56247E-05 | 0.04 | 0.052986905 |
| CAM  | 4.4056E-05  | 0.04 | 0.12684268  |
| SAM  | 3.57676E-06 | 0.21 | 0.046527794 |
| SAS  | 0.000550631 | 0.21 | 0.204864801 |
| SEA  | 6.27064E-05 | 0.04 | 0.08572204  |
| CHI  | 0.000167734 | 0.04 | 0.114203457 |
| NAF  | 2.81278E-07 | 0.04 | 0.038346516 |
| SSA  | 0.000550631 | 0.04 | 0.204864801 |
| SIS  | 0.000426887 | 0.13 | 1.577927496 |"""

In [19]:
def FUND_to_WILIAM(table):
    """
    Converts a table from FUND format to WILIAM format based on a correspondence table.

    Parameters:
    - table: pandas DataFrame
        The input table in FUND format.

    Returns:
    - output_df: pandas DataFrame
        The converted table in WILIAM format.
    """

    correspondance_table = pd.read_csv(r'C:\Users\gabriel.genelot\Documents\damage-functions\WILIAM\geography\correspondance.csv')

    output_df = pd.DataFrame(columns=correspondance_table['Region_WILIAM'].values)

    # Clean the column names
    table.columns = table.columns.str.strip()

    # Check if the table has a 'Region' column, if so, set it as the index and transpose the table to have regions as columns
    if 'Region' in table.columns:
        correct_direction = False
        table['Region'] = table['Region'].str.strip()
        table.set_index('Region', inplace=True)
        table = table.transpose()

    # Iterate over the columns of the output table to get every column translated from FUND to WILIAM regions
    for col in output_df.columns:
        FUND_col = map_regions('Region_WILIAM', 'Region_FUND_code', col)
        try:
            output_df[col] = table[FUND_col].copy()
            output_df[col] = output_df[col].str.split('(')
            output_df[col] = output_df[col].str[0]
            print(f'Added column {col} from {FUND_col}')
        
        except Exception as e:  
            print(f'Could not find values for {col} : {FUND_col}. Error: {e}')
            output_df[col] = 0

    # Get the first column of the table (usually the year, or an important value)
    if correct_direction: 
        first_column = table.columns[0]
        output_df.insert(0, first_column, table[first_column].copy())

    return output_df

test=pd.read_csv(r'C:\Users\gabriel.genelot\Documents\damage-functions\WILIAM\df_parameters\FUND\output\table_ets.csv')


df_output = FUND_to_WILIAM(test)
df_output




Could not find values for nan : Region_WILIAM
NaN    SAS
NaN    WEU
NaN    SIS
NaN    SIS
NaN    SIS
      ... 
NaN    SIS
NaN    SEA
NaN    SIS
NaN    MDE
NaN    CEE
Name: Region_FUND_code, Length: 80, dtype: object. Error: "['CEE'] not in index"
Could not find values for LROW : Region_WILIAM
LROW    CEE
LROW    NAF
LROW    SSA
LROW    FSU
LROW    FSU
       ... 
LROW    MDE
LROW    NAF
LROW    MDE
LROW    SSA
LROW    SSA
Name: Region_FUND_code, Length: 87, dtype: object. Error: "['CEE', nan] not in index"
Could not find values for LROW : Region_WILIAM
LROW    CEE
LROW    NAF
LROW    SSA
LROW    FSU
LROW    FSU
       ... 
LROW    MDE
LROW    NAF
LROW    MDE
LROW    SSA
LROW    SSA
Name: Region_FUND_code, Length: 87, dtype: object. Error: "['CEE', nan] not in index"
Could not find values for nan : Region_WILIAM
NaN    SAS
NaN    WEU
NaN    SIS
NaN    SIS
NaN    SIS
      ... 
NaN    SIS
NaN    SEA
NaN    SIS
NaN    MDE
NaN    CEE
Name: Region_FUND_code, Length: 80, dtype: object. Erro

,NaN,LROW,LROW,NaN,LROW,NaN,LATAM,LROW,NaN,EASOC,...,NaN,LATAM,NaN,NaN,NaN,LROW,LROW,NaN,LROW,LROW
Î±,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Î´,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Î²,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
